In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qap_lp.models import *
from qap_lp.conf import *

# instance_name = 'esc16h' ill conditions
instance_name = 'bur26a'

kwargs = {}
msk_params = {**MSK_DEFAULT, **kwargs}
qap_params = {**QAP_DEFAULT, **kwargs}

#
qap_params['scaling'] = 'l1'
qap_params['mu'] = 0.1
# coefficients
A0, B0 = parse(f'{QAP_INSTANCE}/{instance_name}.dat')

# parse known solution
_, best_obj, arr = parse_sol(f'{QAP_SOL}/{instance_name}.sln')

param = QAPParam(A0, B0, best_obj, arr, **qap_params)

# unpacking
A, B, n, m, e, E, ab = \
    param.A, param.B, param.n, param.m, param.e, param.E, param.ab
# more
nabla = QAPDerivativeL2Penalty(param, qap_params['mu'])
x = x0 = np.ones((n, n)) / n
nabla.obj(param.xo), nabla.obj(x)

(13.42935979608503, 17.187692098294942)

In [12]:
d0 = nabla.partial_f(param.xo)
d0

array([[ 1.71030958e+00,  5.29476107e-01,  1.03878344e+00,
         1.07705214e+00,  5.02900839e+00,  4.21554604e-01,
         8.52518003e-01,  1.23261155e+00,  2.60258111e+00,
         6.46440149e-02,  4.06201589e-01,  1.21080700e+00,
         6.93434631e-01,  2.93305947e+00,  8.74740776e-01,
         2.99467940e-01,  3.73431661e-03,  2.28732956e+00,
         1.76185256e+00,  1.80341756e+00,  1.13212651e+00,
         2.15120394e-01,  3.26355515e-01,  1.91912693e-02,
         2.68628276e-02,  1.10542206e-01],
       [ 1.70930238e+00,  5.80576109e-01,  1.03140884e+00,
         1.17669084e+00,  5.04178525e+00,  4.23074068e-01,
         9.34148333e-01,  1.25481452e+00,  2.69937390e+00,
         6.17882155e-02,  3.97713381e-01,  1.21229677e+00,
         6.92016630e-01,  2.94530921e+00,  6.77596575e-01,
         2.88381301e-01,  3.78381054e-03,  2.29465466e+00,
         1.77757430e+00,  1.89346185e+00,  1.15480957e+00,
         2.22608825e-01,  3.37622807e-01,  1.90353634e-02,
         2.70

In [15]:
lb_x, lb_y = np.where(param.xo <= 1e-4)
lb_x = lb_x.tolist()
lb_y = lb_y.tolist()

In [38]:
dp, m, D, constrs_lb, constrs_a, constrs_b = msk_pd_on_dc_lp(
          param,
          d0,
          (lb_x, lb_y),
      )

In [39]:
a = constrs_a.dual()
b = constrs_b.dual()
l = constrs_lb.dual()

In [42]:
i, j = lb_x[0], lb_y[0]
i, j

(0, 0)

In [40]:
d0[i, j], dp[i,j], a[i], b[j], l[0]

(1.710309584498503, 0.0, -0.0, 0.11054220594422037, 1.5997673785542825)

In [43]:
lb_x.pop(0), lb_y.pop(0)

(0, 0)

In [44]:
dp, m, D, constrs_lb, constrs_a, constrs_b = msk_pd_on_dc_lp(
          param,
          d0,
          (lb_x, lb_y),
      )
a1 = constrs_a.dual()
b1 = constrs_b.dual()
l1 = constrs_lb.dual()

In [45]:
dp[i, j], a[i]+  b[j], a1[i] + b1[j], l[0]

(0.0, 0.11054220594422037, 1.710309584498503, 1.5997673785542825)

#### gradient projection

- projection may be computed by mosek
- or some other methods

#### project point onto $D_n$

- projection may be computed by mosek
- or some other methods

# eval

In [ ]:
x_int, _ = extract_sol_rounding(x, param.A, param.B)

In [ ]:
def check_obj_val(x_sol):
    _obj = A0.T.dot(x_sol).dot(B0).dot(x_sol.T).trace()
    print(f'original obj {_obj}')
    return _obj

In [ ]:
check_obj_val(x)

In [ ]:
nabla.original_obj(xol), nabla.obj(xol, mu)